## Purpose:

Download the raster for a large study area and create average rasters, uncertainty map (a count of data-points used in the average), and standard deviation.

In [3]:
import pandas as pd
import rasterio as rio
import geopandas as gpd
import shapely
import shapely.wkt
import matplotlib.pyplot as plt
import numpy as np
import os

## Divide the Original Study Area into Quarters

In [103]:
study_area = gpd.read_file('../../data/shapefiles/study_area.csv').iloc[0]['geometry']
bounds = study_area.bounds
length = bounds[2] - bounds[0]
width = bounds[3] - bounds[1]

box = shapely.geometry.box(bounds[0], bounds[1], bounds[0]+length/2, bounds[1]+width/2)

with open('../../data/shapefiles/quarter.csv', 'w') as dst:
    dst.write('wkt;\n')
    dst.write(str(box))


## Filter the Index based on the New 'Study Area'

In [104]:
index = gpd.read_file('../../data/shapefiles/ArcticDEM_Strip_Index_Rel7/ArcticDEM_Strip_Index_Rel7.shp')

In [105]:
index = index[['name', 'fileurl', 'acquisitio', 'dx', 'dy', 'dz', 'geometry']]
index = index.set_index('name', drop=True)

In [109]:
def check_geom(strip):
    if strip['geometry'].intersects(box):
        return True
    return False

In [110]:
index = index.loc[index.apply(check_geom, axis='columns')]

In [114]:
out_dir = '../../data/shapefiles/large_study_area/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
index.to_file(out_dir + 'larger_area.shp')

## Loop Over and Create Uncertainty Mask
This will be a count of the number of rasters covering each pixel

In [7]:
index = gpd.read_file('../../data/shapefiles/large_study_area/larger_area.shp')
index.iloc[0]['fileurl']

'http://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/geocell/v3.0/2m/n70w153/SETSM_GE01_20120812_10504100007CE100_1050410000778700_seg1_2m_v3.0.tar.gz'